# Assigment 4

In [1]:
from IPython.display import display, HTML,IFrame
import math
import numpy as np
import scipy.sparse as sp

import igl
import meshplot as mp

from math import sqrt

html=True
if html:
    mp.website()
file_counter = 0
def htmlit(plot,method=1):
    if method==1:
        global file_counter
        unique_filename = f"./htmls/plot_{file_counter}.html"
        file_counter += 1
        plot.save(unique_filename)
        return unique_filename
    else:
        html = plot.to_html(imports=True, html_frame=False)
        return HTML(html)

In [2]:
v, f = igl.read_triangle_mesh("data/irr4-cyl2.off")
tt, _ = igl.triangle_triangle_adjacency(f)

c = np.loadtxt("data/irr4-cyl2.constraints")
cf = c[:, 0].astype(np.int64)
c = c[:, 1:]

In [3]:
def plot_mesh_field(V, F, R, constrain_faces):
    # Highlight in red the constrained faces
    col = np.ones_like(f)
    col[constrain_faces, 1:] = 0
    
    # Scaling of the representative vectors
    avg = igl.avg_edge_length(V, F)/2

    #Plot from face barycenters
    B = igl.barycenter(V, F)

    p = mp.plot(V, F, c=col)
    p.add_lines(B, B + R * avg)

    return p

# 1. Tangent vector fields for scalar field design


In [4]:
def align_field_Q(V, F, TT, soft_id, soft_value, llambda):
    assert (soft_id[0] > 0)
    assert (soft_id.shape[0] == soft_value.shape[0])

    # Edges
    e1 = V[F[:, 1], :] - V[F[:, 0], :]
    e2 = V[F[:, 2], :] - V[F[:, 0], :]

    # Compute the local reference systems for each face, T1, T2
    T1 = e1 / np.linalg.norm(e1, axis=1)[:, None]

    T2 = np.cross(T1, np.cross(T1, e2))
    T2 /= np.linalg.norm(T2, axis=1)[:, None]

    # Arrays for the entries of the matrix
    data = []
    ii = []
    jj = []

    for f in range(F.shape[0]):
        for ei in range(3):  # Loop over the edges

            # Look up the opposite face
            g = TT[f, ei]

            # If it is a boundary edge, it does not contribute to the energy
            # or avoid to count every edge twice
            if g == -1 or f > g:
                continue

            # Compute the complex representation of the common edge
            e = V[F[f, (ei + 1) % 3], :] - V[F[f, ei], :]

            vef = np.array([np.dot(e, T1[f, :]), np.dot(e, T2[f, :])])
            vef /= np.linalg.norm(vef)
            ef = (vef[0] + vef[1] * 1j).conjugate()

            veg = np.array([np.dot(e, T1[g, :]), np.dot(e, T2[g, :])])
            veg /= np.linalg.norm(veg)
            eg = (veg[0] + veg[1] * 1j).conjugate()

            # Add the term conj(f)^n*ui - conj(g)^n*uj to the energy matrix
            data.append(np.dot(ef,ef.conjugate()))
            ii.append(f)
            jj.append(f)

            data.append(np.dot(ef.conjugate(),-eg))
            ii.append(f)
            jj.append(g)

            data.append(np.dot(-ef, eg.conjugate()))
            ii.append(g)
            jj.append(f)

            data.append(np.dot(eg.conjugate(), eg))
            ii.append(g)
            jj.append(g)


    # Convert the constraints into the complex polynomial coefficients and add them as soft constraints
    Q = sp.coo_matrix((data, (ii, jj)), shape=(F.shape[0], F.shape[0])).asformat("lil")

    # Rhs of the system
    b = np.zeros(F.shape[0], dtype=complex)


    for ci in range(soft_id.shape[0]):
        f = soft_id[ci]
        v = soft_value[ci, :]
        # Project on the local frame
        c = np.dot(v, T1[f, :]) + np.dot(v, T2[f, :]) * 1j
        Q[f,:]=0
        Q[f,f]=1
        b[f]=c

    for ci in range(soft_id.shape[0]):
        f = soft_id[ci]
        v = soft_value[ci, :]

        # Project on the local frame
        c = np.dot(v, T1[f, :]) + np.dot(v, T2[f, :]) * 1j

        for fj in range(F.shape[0]):
            b[fj] -= np.dot(Q[fj,f],c)

    #assert (b.shape[0] == index)

    # Solve the linear system
    A = Q.asformat("csr")
    u = sp.linalg.spsolve(A, b)
    for ci in range(soft_id.shape[0]):
        f = soft_id[ci]
        v = soft_value[ci, :]
        # Project on the local frame
        c = np.dot(v, T1[f, :]) + np.dot(v, T2[f, :]) * 1j
        u[f] = c

    R = T1 * u.real[:, None] + T2 * u.imag[:, None]

    return R

In [5]:
R2 = align_field_Q(v, f, tt, cf, c, 1e6)
p=plot_mesh_field(v, f, R2, cf)
np.savetxt("1_interpolated_field.txt",R2)
if html:
    html=htmlit(p)
IFrame(src=html, width=700, height=600)

Plot saved to file ./htmls/plot_0.html.


# 2. Reconstructing a scalar field from a vector field

In [6]:
def scalar_field(v,f,vector_field):
    G=igl.grad(v,f)
    area=igl.doublearea(v,f)/2
    At=np.diag(np.tile(area, 3))
    ut=vector_field.T.flatten()
    K=G.T@At@G
    b=G.T@At@ut
    '''
    K[0,:]=0
    K[0,0]=1
    b[0]=0
    '''
    si=sp.linalg.spsolve(K, b)
    gt=(G@si).reshape(3,-1).T
    poisson_error1=np.linalg.norm(gt-vector_field,axis=1)
    poisson_error2=np.linalg.norm(gt-vector_field)

    return si,poisson_error1,poisson_error2,gt


def plot_mesh_scalar_field(V, F, R, s_field):
    avg = igl.avg_edge_length(V, F) / 2
    B = igl.barycenter(V, F)
    p = mp.plot(V, F, c=s_field)

    p.add_lines(B, B + R * avg)
    return p

In [7]:
print("Below is visualization of computed scalar function and its gradient.")
s_field,p_error1,p_error2,gt = scalar_field(v,f,R2)
p=plot_mesh_scalar_field(v,f,gt,s_field)

np.savetxt("2_reconstructed_scalar_function.txt",s_field)
if html:
    html=htmlit(p)
IFrame(src=html, width=700, height=600)

Below is visualization of computed scalar function and its gradient.
Plot saved to file ./htmls/plot_1.html.


F:\tools\miniconda\envs\gp\lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:214: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',


In [8]:
def plot_possion_error(V,F,error):
    p=mp.plot(V,F,c=error)
    return p

In [9]:
print("Below is plot of the Poisson reconstruction error.")
p=plot_possion_error(v,f,p_error1)
if html:
    html=htmlit(p)
IFrame(src=html, width=700, height=600)

Below is plot of the Poisson reconstruction error.
Plot saved to file ./htmls/plot_2.html.


In [10]:
print("The poisson reconstruction error is: ")
print(p_error2)

The poisson reconstruction error is: 
7.375992424336967


# 3. Harmonic and LSCM Parameterizations

In [11]:
v, f = igl.read_triangle_mesh("data/camel_head.off")

## Harmonic

In [12]:
def harmonic_para(v,f):
    boundaryloop=igl.boundary_loop(f)
    boundaryuv=igl.map_vertices_to_circle(v,boundaryloop)
    uv = igl.harmonic(v,f,boundaryloop, boundaryuv,1)
    return uv

In [13]:
print("Harmonic: The first plot is a visualization of the computed mapping functions. The second plot is a visualization of the flattened mesh on the UV plane. The third plot is the gradient of its V function.")
uv=harmonic_para(v,f)
G = igl.grad(v, f)
v_grad = (G @ uv[:, 1]).reshape(3, -1).T
p = mp.plot(v, f, uv=uv)
if html:
    html=htmlit(p)
IFrame(src=html, width=700, height=600)

Harmonic: The first plot is a visualization of the computed mapping functions. The second plot is a visualization of the flattened mesh on the UV plane. The third plot is the gradient of its V function.
Plot saved to file ./htmls/plot_3.html.


In [14]:
p=mp.plot(uv, f, shading={"wireframe": True})
if html:
    html=htmlit(p)
IFrame(src=html, width=700, height=600)

Plot saved to file ./htmls/plot_4.html.


In [15]:
p=plot_mesh_scalar_field(v,f,v_grad,uv[:,1])
if html:
    html=htmlit(p)
IFrame(src=html, width=700, height=600)

Plot saved to file ./htmls/plot_5.html.


## LSCM

In [16]:
def LSCM_para(v,f):
    boundaryloop = igl.boundary_loop(f)
    b = np.array([boundaryloop[0], boundaryloop[int(len(boundaryloop) / 2)]])
    bc = np.array([[0., 0.], [1., 0.]])
    _,uv = igl.lscm(v, f,b,bc)
    return uv

In [17]:
print("LSCM: The first plot is a visualization of the computed mapping functions. The second plot is a visualization of the flattened mesh on the UV plane. The third plot is the gradient of its V function.")
uv=LSCM_para(v,f)
G = igl.grad(v, f)
v_grad = (G @ uv[:, 1]).reshape(3, -1).T
p = mp.plot(v, f, uv=uv)
if html:
    html=htmlit(p)
IFrame(src=html, width=700, height=600)


LSCM: The first plot is a visualization of the computed mapping functions. The second plot is a visualization of the flattened mesh on the UV plane. The third plot is the gradient of its V function.
Plot saved to file ./htmls/plot_6.html.


In [18]:
p=mp.plot(uv, f, shading={"wireframe": True})
if html:
    html=htmlit(p)
IFrame(src=html, width=700, height=600)

Plot saved to file ./htmls/plot_7.html.


In [19]:
p=plot_mesh_scalar_field(v,f,v_grad,uv[:,1])
if html:
    html=htmlit(p)
IFrame(src=html, width=700, height=600)

Plot saved to file ./htmls/plot_8.html.


# 4. Editing a parameterization with vector fields

In [20]:
print("The first plot is a virtualization of harmonic parameteraizaion before editting. The second plot is that after editting.")
v, f = igl.read_triangle_mesh("data/irr4-cyl2.off")
tt, _ = igl.triangle_triangle_adjacency(f)
c = np.loadtxt("data/irr4-cyl2.constraints")
cf = c[:, 0].astype(np.int64)
c = c[:, 1:]
R = align_field_Q(v, f, tt, cf, c, 1e6)
s_field,_,_,_ = scalar_field(v,f,R)
uv=harmonic_para(v,f)
p=mp.plot(v, f,uv=uv, shading={"wireframe": False})
if html:
    html=htmlit(p)
IFrame(src=html, width=700, height=600)

The first plot is a virtualization of harmonic parameteraizaion before editting. The second plot is that after editting.
Plot saved to file ./htmls/plot_9.html.


In [21]:
uv_e=uv.copy()
uv_e[:,1]=s_field

p = mp.plot(v, f, uv=uv_e)
#mp.plot(uv_e, f, c=color,shading={"wireframe": True})
if html:
    html=htmlit(p)
IFrame(src=html, width=700, height=600)

Plot saved to file ./htmls/plot_10.html.


In [22]:
print("The first plot is a visualization of flipped elements within 3d space of xyz. The second plot is a visualization of flipped elements within 2d space of UV")
color=np.ones_like(f)
flipped_face=[]
for i,face in enumerate(f):
    #print(uv_e[face[1]])
    if np.cross(uv_e[face[1]]-uv_e[face[0]],uv_e[face[2]]-uv_e[face[0]])>0:
        color[i]=np.array([1.0,0.0,0.0])
        flipped_face.append(i)
p=mp.plot(v, f, c=color,shading={"wireframe": True})
if html:
    html=htmlit(p)
IFrame(src=html, width=700, height=600)

The first plot is a visualization of flipped elements within 3d space of xyz. The second plot is a visualization of flipped elements within 2d space of UV
Plot saved to file ./htmls/plot_11.html.


In [23]:
p=mp.plot(uv_e, f, c=color,shading={"wireframe": True})
if html:
    html=htmlit(p)
IFrame(src=html, width=700, height=600)

Plot saved to file ./htmls/plot_12.html.


In [24]:
np.savetxt("4_flipped_triangle_indices.txt",flipped_face,fmt="%d")